# Глубокий кросс-энтропийный метод

#### дедлайн (жёсткий) задания: 13 марта, 23:59 UTC+3

#### при сдаче задания нужно данный файл и сгенерированные в результате его работы файлы и директории поместить в архив, сохраняя относительные пути, и послать архив в систему сдачи

В данной работе будет рассмотрено обобщение кросс-энтропийного метода на случай параметризации агента с помощью нейросети. Перед вами будет поставлена задача обучить многослойную нейронную сеть для решения простых игр в непрерывных пространствах состояний.

# Работа выполнена: ФИО, номер группы.

In [ ]:
# Для корректной работы ноутбука может понадобиться исполнение следующих команд:
# (инструкция для UNIX-подобных систем):
# !apt-get install -y xvfb x11-utils ffmpeg libav-tools python-opengl swig
# !pip install --upgrade piglet pyvirtualdisplay joblib box2d-py gym[all]==0.18.3
# (команда для MacOS системы):
# !brew install XQuartz swig # для использования xvfb в MacOS сервере (не опробовано)

Для визуализации среды на headless-сервере с UNIX-подобной ОС (например, Google Colab) раскомментируйте следующую ячейку:

In [ ]:
# from pyvirtualdisplay import Display # для визуализации на UNIX-подобном сервере


# virtual_display = Display(visible=0, size=(1920, 1080))
# virtual_display.start()

В качестве альтернативы предыдущей ячейке можно раскомментировать ячейку ниже:

In [ ]:
# XVFB будет запущен в случае исполнения на сервере, если раскомментировать код ниже:
# import os
# if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
#    !bash ../xvfb start
#    os.environ['DISPLAY'] = ':1'

Скрыть предупреждения: эстетическая опция, можно не выполнять ячейку ниже:

In [ ]:
import warnings


warnings.filterwarnings("ignore")

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline
sns.set(font_scale=1.)

# если есть вывод "<classname> has no attribute .env", удалите .env или обновите gym
env = gym.make("CartPole-v1").env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]

plt.imshow(env.render("rgb_array"))
print("размерность вектора состояний dim =", state_dim)
print("n_actions =", n_actions)

env.close()

In [ ]:
env.observation_space, env.action_space

# Нейросетевая политика, основная задача (10 баллов)

Для текущей задачи будет использована упрощённая нейронная сеть, реализованная в библиотеке __[Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)__. Что для решения задачи понадобится:

* `agent.partial_fit(states, actions)` - выполнение одного прохода (одной эпохи) по данным для настройки параметров. В ходе вызова данного метода происходит приближённая максимизация вероятности :actions: при условии :states:
* `agent.predict_proba(states)` - оценка вероятностей всех действий, матрица формы __[len(states), n_actions]__


In [ ]:
from sklearn.neural_network import MLPClassifier, MLPRegressor


agent = MLPClassifier(
    hidden_layer_sizes=(20, 20),
    activation='tanh',
)

# инициализация агента на размерности пространства состояний и пространства действий
agent.partial_fit([env.reset()] * n_actions, range(n_actions), range(n_actions))

In [ ]:
def generate_session(agent, t_max=1000, test=False):
    """
    Сыграть отдельный эпизод, используя нейросетевую параметризацию агента.
    Останов после :t_max: шагов среды.
    """
    states, actions = [], []
    total_reward = 0

    s = env.reset()

    for t in range(t_max):
        
        # Используйте модель агента для оценки распределения на действия для текущего состояния :s:
        probs = <Код для вывода оценки вероятности по выходу нейросети>

        assert probs.shape == (n_actions,), "Нужно получить вектор вероятностей (функция np.reshape в помощь)"
        
        # Используйте текущую оценку политики для выбора действия
        if test:
            # на тестовом прогоне или на валидации используйте
            # детерминированную стратегию
            a = <Выбор наиболее вероятного действия>
            # ^-- подсказка: попробуйте функцию np.argmax
        else:
            # сэмплирование пропорционально политике $\pi(a|s)$,
            # не нужно выбирать детерминированно наиболее вероятное действие
            a = <Сэмплирование действия относительно распределения, задаваемого политикой>
            # ^-- подсказка: попробуйте функцию np.random.choice

        new_s, r, done, info = env.step(a)

        # Запись статистики текущего эпизода
        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done:
            break
    return states, actions, total_reward

In [ ]:
dummy_states, dummy_actions, dummy_reward = generate_session(agent, t_max=5)
print("states:", np.stack(dummy_states))
print("actions:", dummy_actions)
print("reward:", dummy_reward)

### Шаги кросс-энтропийного метода
Общая схема кросс-энтропийного метода (CEM) приведена на странице 35 (параграф 2.2.4.) учебного [пособия](https://arxiv.org/pdf/2201.09746.pdf).

Глубокий CEM использует точно такую же стратегию, что и обычный CEM.

Главное отличие состоит в том, что теперь каждое наблюдение не число, а `float32` вектор.

In [ ]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Выбрать states и actions из игр с rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, оба 1D lists of states и actions из наилучших эпизодов

    Просьба сохранять порядок elite states и actions 
    [то есть сортированы по номерам эпизодов и в хронологическом порядке в каждом эпизоде]

    Просьба не считать по умолчанию states как целочисленные значения
    (они позже примут другой формат).
    """

    <Ваша имплементация>
        
    # Заметим, что это отличается от табличного случая.
    # Теперь наша нейронная сеть обучается на одном объекте с входной размерностью `(1, n_states)`.
    # То есть следует использовать `np.squeeze`, чтобы избавиться от ведущего измерения в `select_elites`.
    
    return elite_states, elite_actions

# Цикл обучения
Генерация эпизодов, выбор N лучших и обучение на них.

In [ ]:
from IPython.display import clear_output


def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    Функция-помощник, которая визуализирует процесс обучения.
    Никакой крутой математики здесь нет, только построение графиков.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])

    print("средняя награда = %.3f, порог=%.3f" % (mean_reward, threshold))
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Средние награды')
    plt.plot(list(zip(*log))[1], label='Пороги наград')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines([np.percentile(rewards_batch, percentile)],
               [0], [100], label="перцентиль", color='red')
    plt.legend()
    plt.grid()

    clear_output(True)
    plt.show()

In [ ]:
%%time

sns.set(font_scale=1.)
n_sessions = 100
percentile = 70
log = []

for i in range(100):
    print("Итерация метода № {}:".format(i + 1))
    # генерация новых эпизодов
    sessions = [<генерация списка из n_sessions эпизодов>]

    states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))

    elite_states, elite_actions =  <выбор наилучшых действий с помощью select_elites()>

    <partial_fit агента для обучения на elite_actions (y, целевой переменной)
     при условии elite_states (X, данных)>

    show_progress(rewards_batch, log, percentile, reward_range=[0, np.max(rewards_batch)])

    if np.mean(rewards_batch) > 190:
        print("Вы выиграли! Можете прервать процедуру обучения с помощью сигнала KeyboardInterrupt.")

# Результаты

In [ ]:
# запись эпизодов
import gym.wrappers


for kind, directory in [(True, "test"), (False, "sample")]:
    env = gym.wrappers.Monitor(gym.make("CartPole-v1"),
                               directory="videos/CartPole-v1/{}".format(directory), force=True)
    sessions = [generate_session(<Попробуйте поварьировать значение test = {TRUE, FALSE}>) for _ in range(100)]
    env.close()

Детерминированная политика:

In [ ]:
# демонстрация видео
from IPython.display import HTML
import os


video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/CartPole-v1/test")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/CartPole-v1/test/"+video_names[-1]))

Стохастическая политика:

In [ ]:
video_names = list(
    filter(lambda s: s.endswith(".mp4"), os.listdir("./videos/CartPole-v1/sample")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/CartPole-v1/sample/"+video_names[-1]))

# Дополнительные задания

### Глубокий кросс-энтропийный метод

На данный момент у Вас должна быть достаточно хорошая награда в [CartPole-v1](https://www.gymlibrary.dev/environments/classic_control/cart_pole/), чтобы считать основное задание выполненным (см. ссылку). Время попробовать что-нибудь посложнее.

### Задания (вплоть до 5 баллов)

* __2.1__ Выбрать следующие среды: MountainCar-v0 и LunarLander-v2.
  * Для MountainCar, получить среднее вознаграждение __как минимум -150__
  * For LunarLander, получить среднее вознаграждение __как минимум +50__

Рекомендуем обратить внимание на раздел с советами ниже, это важно.

* __2.2__ Изучить зависимость скорости обучения агента от гипрепараметров алгоритма в среде MountainCar-v0. Постараться продемонстрировать возможность ускорения обучения хотя бы в два раза при грамотном подборе гиперпараметров
  * Очевидное улучшение: использовать [joblib](https://joblib.readthedocs.io/en/stable/) или multiprocessing
  * Попробовать переиспользовать сэмплы из 3-5 последних итераций при вычислении порога и при обучении
  * Поэкспериментировать с количеством итераций обучения и шагом метода обучения (learning rate) нейронной сети (смотреть params)

* __При сдаче задания требуется перечислить в данном файле, что было сделано и чего добились__
  
### Советы
* Страница Gym: [MountainCar](https://www.gymlibrary.dev/environments/classic_control/mountain_car/), [LunarLander](https://www.gymlibrary.dev/environments/box2d/lunar_lander/)
* Эпизоды в MountainCar могут длиться более 10k+ шагов. Убедитесь, что параметр ```t_max``` не меньше 10k.
 * Также может быть хорошей идеей строго отбирать награды через ">", а не ">=" (strictly_select_elites). Если 90% Ваших эпизодов имеют награду -10k и 20% лучше, тода если Вы используйте перцентиль 20% как порог, R >= порог __не может отклонить неуспешные эпизоды__, в то время как R > справляется с этим корректно.
* _проблема сред gym_: некоторые версии gym ограничивают эпизод 200 шагами. Это ограничивает возможности CEM в обучении в большинстве случаев. Убедитесь, что Ваш агент способен симулировать эпизоды с заданным __t_max__, и если нет, то попробуйте `env = gym.make("MountainCar-v0").env` или в ином случае избавьтесь от TimeLimit wrapper.
* Если Вы пользуетесь старой _swig_ библиотекой для LunarLander-v2, у Вас может возникнуть ошибка. Детали по [ссылке](https://github.com/openai/gym/issues/100) с решением проблемы.
* Если CEM не будет обучаться, то построение диаграмм распределения награды и запись видео эпизодов могут помочь: они Вас могут натолкнуть на идею исправления.
* 20-нейронной сети может не хватить, не стесняйтесь экспериментировать.

Ячейка с кодом ниже может оказаться полезной:

In [ ]:
def visualize_mountain_car(env, agent):
    xs = np.linspace(env.min_position, env.max_position, 100)
    vs = np.linspace(-env.max_speed, env.max_speed, 100)
    grid = np.dstack(np.meshgrid(xs, vs)).transpose(1, 0, 2)
    grid_flat = grid.reshape(len(xs) * len(vs), 2)
    probs = agent.predict_proba(grid_flat).reshape(len(xs), len(vs), 3)
    return probs

Функции-помощники для инициализации среды:

In [ ]:
def get_env(name, classification=True):
    env = gym.make(name).env
    
    env.reset()
    if classification:
        n_actions = env.action_space.n
    else:
        n_actions = sum(env.action_space.shape)
    state_dim = env.observation_space.shape[0]
    
    plt.imshow(env.render("rgb_array"))
    print("state vector dim =", state_dim)
    print("n_actions =", n_actions)
    
    env.close()
    return env, n_actions, state_dim

Строгий отбор states и actions на основе перцентилей:

In [ ]:
def strictly_select_elites(states_batch, actions_batch, rewards_batch, percentile=50, classification=True):
    """
    Выбрать states и actions из игр с rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, оба 1D lists of states и actions из наилучших эпизодов

    Просьба сохранять порядок elite states и actions 
    [то есть сортированы по номерам эпизодов и в хронологическом порядке в каждом эпизоде]

    Просьба не считать по умолчанию states как целочисленные значения
    (они позже примут другой формат).
    Отбор в случае решения задачи классификации и в случае решения задачи регрессии несколько отличаются.
    """

    < Ваша имплементация >
    
    return elite_states, elite_actions

Обёртка для инициализации агента:

In [ ]:
from sklearn.neural_network import MLPRegressor


def init_agent(env, classification=True, **params):
    if classification:
        agent = MLPClassifier(
            hidden_layer_sizes=(20, 20, 20, 20),
            activation='tanh',
        )
    else:
        agent = MLPRegressor(
            hidden_layer_sizes=(20, 20, 20, 20),
            activation='tanh',
        )
    
    agent.set_params(**params)
    
    if classification:
        n_actions = env.action_space.n
        agent.partial_fit([env.reset()] * n_actions, range(n_actions), range(n_actions))
    else:
        n_actions = sum(env.action_space.shape)
        if n_actions > 1:
            agent.partial_fit([env.reset()], np.random.randn(1, n_actions))
        else:
            agent.partial_fit([env.reset()], np.random.randn(n_actions))
    
    return agent

Генератор эпизодов, приспособленный к параллелизации:

In [ ]:
import pickle as pkl


def generate_parallel_session(agent, name='MountainCar-v0', t_max=10000, classification=True,
                              epsilon=0.01, agent_mul_fac=1.0, test=False, env=None):
    """
    Сыграть отдельный эпизод, используя нейросетевую параметризацию агента.
    Останов после :t_max: шагов среды.
    """
    
    states, actions = [], []
    total_reward = 0
    
    if env is None:
        env = gym.make(name).env
    if classification:
        n_actions = env.action_space.n
    else:
        n_actions = sum(env.action_space.shape)
    s = env.reset()

    for t in range(t_max):
        
        if classification:
            probs = < оценка вероятностей нейросетевой модели >

            assert probs.shape == (n_actions,), "Нужно получить вектор вероятностей" +\
                "(функция np.reshape в помощь)"
            
            if test:
                a = < наиболее вероятное действия относительно probs >
            else:
                a = < сэмпл действия из распределения probs >
        else:
            expected_action = < оценка с помощью модели среднего действия, помноженного на agent_mul_fac >
            
            if test:
                a = < само expected_action >
            else:
                a = < сэмпл из Normal(expected_action, epsilon) >

        new_s, r, done, info = env.step(a)

        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done:
            break
    
    del env, s, new_s, a
    
    return states, actions, total_reward

Функция для обучения агента:

In [ ]:
from collections import deque
# from multiprocessing import Pool
from joblib import Parallel, delayed
from functools import partial


def train_agent(classification=True, epsilon=0.01, name='MountainCar-v0', n_train_steps=100,
                n_game_steps=10000, n_sessions=100, percentile=70, goal_score=-150, history_length=4,
                n_jobs=16, verbose=True, agent_mul_fac=1.0, **params):
    env = gym.make(name).env
    if classification:
        n_actions = env.action_space.n
    else:
        n_actions = sum(env.action_space.shape)
    
    agent = init_agent(env, classification, **params)
    
    if verbose:
        print('Шаг агента = {}'.format(agent.learning_rate_init), flush=True)
    
    log = []
    sessions = deque([], history_length * n_sessions)
    
    for i in range(n_train_steps):
        if n_jobs > 1:
            # pool = Pool(processes=n_jobs, maxtasksperchild=10)
            # sessions.extend(pool.map(partial(generate_parallel_session, agent, name, n_game_steps,
            #                                  classification, epsilon, agent_mul_fac), [False] * n_sessions))
            # pool.close()
            # pool.join()
            # del pool
            sessions.extend(Parallel(n_jobs=n_jobs)(delayed(partial(generate_parallel_session, agent, name,
                                                                    n_game_steps, classification,
                                                                    epsilon, agent_mul_fac))(x)\
                                                    for x in [False] * n_sessions))
        else:
            sessions.extend([generate_parallel_session(
                agent, name, n_game_steps, classification, epsilon, agent_mul_fac,
                False) for _ in range(n_sessions)])
        
        states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))
        elite_states, elite_actions = strictly_select_elites(
            states_batch, actions_batch, rewards_batch, percentile, classification)
        
        if classification:
            agent.partial_fit(elite_states, elite_actions)
        else:
            elite_actions = elite_actions if (len(elite_actions.shape) < 2) or (elite_actions.shape[1] > 1)\
                            else elite_actions.reshape(-1)
            agent.partial_fit(elite_states, elite_actions / agent_mul_fac)
        
        if verbose:
            inter_min = np.min(rewards_batch)
            min_lim = -n_game_steps if -n_game_steps < inter_min else inter_min
            inter_max = np.max(rewards_batch)
            max_lim = goal_score if goal_score > inter_max else inter_max
            show_progress(rewards_batch, log, percentile, reward_range=[min_lim, max_lim])
            
            if np.mean(rewards_batch) > goal_score:
                print("Вы выиграли! Можете прервать процедуру обучения с помощью сигнала KeyboardInterrupt.")
    
    if verbose:
        mean_reward = np.mean(rewards_batch)
        threshold = np.percentile(rewards_batch, percentile)
        print("средняя награда = %.3f, порог=%.3f" % (mean_reward, threshold))
        del mean_reward, threshold
    
    del env, sessions, states_batch, actions_batch, rewards_batch, elite_states, elite_actions
    
    return agent, log

Функция для постановки экспериментов, зависящих от набора гиперпараметров:

In [ ]:
from time import time


def training_experiment(n_jobs_list, history_length_list, learning_rate_init_list, n_train_steps_list,
                        n_samples=5, name='MountainCar-v0', n_game_steps=10000, n_sessions=100, percentile=70,
                        goal_score=-150, **params):
    experiment_data = []
    
    for n_jobs in n_jobs_list:
        for history_length in history_length_list:
            for learning_rate_init in learning_rate_init_list:
                params['learning_rate_init'] = learning_rate_init
                for n_train_steps in n_train_steps_list:
                    elapsed_time_list = []
                    log_list = []
                    
                    for i in range(n_samples):
                        print(
                            'Запуск: n_jobs = {}, history_length = {},\n'.format(
                                n_jobs, history_length) +\
                            '                  learning_rate_init = {}, n_train_steps = {};\n'.format(
                                learning_rate_init, n_train_steps) +\
                            'сэмпл {} из {}.'.format(
                                i + 1, n_samples), flush=True)
                        
                        elapsed_time = time()
                        
                        agent, log = train_agent(
                            classification=True, epsilon=0.01, name=name, n_train_steps=n_train_steps,
                            n_game_steps=n_game_steps, n_sessions=n_sessions, percentile=percentile,
                            goal_score=goal_score, history_length=history_length, n_jobs=n_jobs, verbose=True,
                            **params)
                        
                        elapsed_time_list.append(time() - elapsed_time)
                        
                        log_list.append(log)
                        
                        del log, agent
                        
                    results = {
                               'name': name,
                               'goal_score': goal_score,
                               'n_jobs': n_jobs,
                               'elapsed_time_list': elapsed_time_list,
                               'history_length': history_length,
                               'learning_rate_init': learning_rate_init,
                               'n_train_steps': n_train_steps,
                               'log_list': log_list
                              }
                        
                    experiment_data.append(results)
                    
                    del elapsed_time_list, log_list
    return experiment_data

Функция-помощник для визуализации эффектов от переиспользования сэмплов:

In [ ]:
def plot_history_length_experiment_results(df_data):
    history_length_vals = []
    train_step_vals = []
    reward_vals = []
    mean_threshold_vals = []
    
    for config in df_data:
        for log in config['log_list']:
            mean_rewards, reward_thresholds = zip(*log)
            mean_rewards, reward_thresholds = list(mean_rewards), list(reward_thresholds)
            reward_vals.extend(mean_rewards)
            reward_vals.extend(reward_thresholds)
            train_steps_num = len(mean_rewards)
            train_step_vals.extend(range(train_steps_num))
            train_step_vals.extend(range(train_steps_num))
            history_length_vals.extend([config['history_length']] * train_steps_num * 2)
            mean_threshold_vals.extend(['mean'] * train_steps_num)
            mean_threshold_vals.extend(['threshold'] * train_steps_num)
    
    df = pd.DataFrame({'Переиспользованных шагов': history_length_vals, 'Шаг №': train_step_vals,
                       'Награда': reward_vals, 'Вид награды': mean_threshold_vals})
    last_steps_count = np.sort(df.loc[:, 'Переиспользованных шагов'].unique())
    max_n_colors = last_steps_count.size
    palette = dict(zip(last_steps_count, sns.hls_palette(max_n_colors, l=.45, s=.8)))
    
    sns.set(font_scale=1.35)
    g = sns.relplot(x='Шаг №', y='Награда', hue='Переиспользованных шагов', style='Вид награды', kind='line',
                    data=df, height=8, aspect=1.5, palette=palette)
    g.fig.suptitle('Влияние количества переиспользованных шагов при обучении {}'.format(df_data[0]['name']))
    clear_output(True)
    plt.show()

Функция-помощник для визуализации влияния шага обучения и количества итераций при обучении агента:

In [ ]:
def plot_lrt_experiment_results(df_data):
    learning_rate_vals = []
    max_train_steps_vals = []
    train_step_vals = []
    reward_vals = []
    mean_threshold_vals = []
    
    for config in df_data:
        for log in config['log_list']:
            mean_rewards, reward_thresholds = zip(*log)
            mean_rewards, reward_thresholds = list(mean_rewards), list(reward_thresholds)
            reward_vals.extend(mean_rewards)
            reward_vals.extend(reward_thresholds)
            train_steps_num = len(mean_rewards)
            train_step_vals.extend(range(train_steps_num))
            train_step_vals.extend(range(train_steps_num))
            learning_rate_vals.extend([config['learning_rate_init']] * train_steps_num * 2)
            max_train_steps_vals.extend([config['n_train_steps']] * train_steps_num * 2)
            mean_threshold_vals.extend(['mean'] * train_steps_num)
            mean_threshold_vals.extend(['threshold'] * train_steps_num)
                
    df = pd.DataFrame({'Шаг метода': learning_rate_vals, 'Максимум шагов': max_train_steps_vals,
                       'Шаг №': train_step_vals, 'Награда': reward_vals,
                       'Вид награды': mean_threshold_vals})
    lr_vals = np.sort(df.loc[:, 'Learning rate'].unique())
    max_n_colors = lr_vals.size
    palette = dict(zip(lr_vals, sns.hls_palette(max_n_colors, l=.45, s=.8)))
    
    sns.set(font_scale=1.35)
    g = sns.relplot(x='Шаг №', y='Награда', hue='Шаг метода', style='Вид награды', kind='line',
                    row='Максимум шагов', data=df, height=6, aspect=1.8, palette=palette)
    g.fig.suptitle('Влияние шага метода и количества итераций на {}'.format(df_data[0]['name']),
                   x=0.8, y=1.05)
    leg = g._legend
    for lr, label in zip(lr_vals, leg.texts[1:]):
        label.set_text("{:.4f}".format(lr))
    clear_output(True)
    plt.show()

Функция для записи сэмплированных игровых эпизодов:

In [ ]:
def record_sessions(agent, name, t_max=10000, classification=True, epsilon=0.01, agent_mul_fac=1.0):
    for kind, directory in [(True, "test"), (False, "sample")]:
        env = gym.wrappers.Monitor(gym.make(name),
                                   directory="videos/{}/{}".format(name, directory), force=True)
        sessions = [generate_parallel_session(agent, name=name, t_max=t_max, classification=classification,
                                              epsilon=epsilon, agent_mul_fac=agent_mul_fac, test=kind,
                                              env=env) for _ in range(100)]
        env.close()

Функция, позволяющая вставить запись игрового эпизода в ноутбук:

In [ ]:
def show_video(name, directory):
    path = "./videos/{}/{}".format(name, directory)
    video_names = list(
        filter(lambda s: s.endswith(".mp4"), os.listdir(path)))
    return HTML("""
    <video width="640" height="480" controls>
    <source src="{}" type="video/mp4">
    </video>
    """.format(path + "/" + video_names[-1]))

Сетка параметров для проведения экспериментов:

In [ ]:
< n_jobs_list = сетка на количество процессов
history_length_list = сетка на размер истории
learning_rate_init_list = сетка на шаги метода
n_train_steps_list = сетка на количество итераций метода >

## Глубокий кросс-энтропийный метод в среде MountainCar-v0

In [ ]:
sns.set(font_scale=1.)
env, n_actions, _ = get_env("MountainCar-v0")

In [ ]:
env.observation_space, env.action_space

Поиск оптимального количества параллельных легковесных процессов (нитей) на конечной машине:

In [ ]:
%%time

n_jobs_experiment_data = < запуск перебора на сетке с помощью training_experiment >

sns.set(font_scale=1.)
pkl.dump(n_jobs_experiment_data, open('MountainCar-v0_n_jobs_experiment_data.pkl', 'wb'))
clear_output(True)

Визуализация результатов эксперимента:

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd


n_jobs_vals = []
elapsed_time_vals = []
elapsed_time_vars = []

for config in n_jobs_experiment_data:
    n_jobs_vals.append(config['n_jobs'])
    elapsed_time_vals.append(np.mean(config['elapsed_time_list']))
    elapsed_time_vars.append(np.std(config['elapsed_time_list']))

df = pd.DataFrame({'Количество нитей': n_jobs_vals, 'Средний ETA, сек.': elapsed_time_vals,
                   'Стандартное отклонение ETA': elapsed_time_vars})

plt.figure(figsize=(12, 6))
sns.set(font_scale=1.35)
plt.title('Производительность 5 итераций обучения на {}'.format(n_jobs_experiment_data[0]['name']),
          fontsize=16)
ax = sns.scatterplot(x='Количество нитей', y='Средний ETA, сек.', size='Стандартное отклонение ETA',
                     sizes=(40, 400), data=df)
min_val = np.min(elapsed_time_vals)
ax.axhline(min_val, color='red', ls='--', linewidth=2, label='{:.2f} сек.'.format(min_val))
plt.legend()
clear_output(True)
plt.show()

Исследование влияния переиспользования сэмплов в процессе обучения:

In [ ]:
%%time

history_length_experiment_data = < запуск перебора на сетке с помощью training_experiment >

sns.set(font_scale=1.)
pkl.dump(history_length_experiment_data, open('MountainCar-v0_history_length_experiment_data.pkl', 'wb'))
clear_output(True)

Результаты текущего эксперимента:

In [ ]:
plot_history_length_experiment_results(history_length_experiment_data)

Поиск оптимального шага обучения для многослойного перцептрона (MLP) вместе c подбором количества шагов обучения:

In [ ]:
%%time

lrt_experiment_data = < запуск перебора на сетке с помощью training_experiment >

sns.set(font_scale=1.)
pkl.dump(lrt_experiment_data, open('MountainCar-v0_lrt_experiment_data.pkl', 'wb'))
clear_output(True)

Результаты текущего эксперимента:

In [ ]:
plot_lrt_experiment_results(lrt_experiment_data)

Обучение агента на задаче MountainCar-v0 с использованием подобранных ранее гиперпараметров:

In [ ]:
%%time

sns.set(font_scale=1.)
agent, _ = < запуск функции train_agent для обучения агента >

Визуализация действий обученного агента в зависимости скорости и положения:

In [ ]:
sns.set(font_scale=1.)
plt.imshow(visualize_mountain_car(env, agent))

Сэмпл детерминированной стратегии по въезду на холм:

In [ ]:
record_sessions(agent, "MountainCar-v0")
show_video("MountainCar-v0", "test")

Сэмпл стохастической стратегии по въезду на холм:

In [ ]:
show_video("MountainCar-v0", "sample")

## Обучение агента в среде LunarLander-v2

In [ ]:
sns.set(font_scale=1.)
env, n_actions, _ = get_env("LunarLander-v2")

In [ ]:
env.observation_space, env.action_space

Обучение агента:

In [ ]:
%%time

agent, _ = < запуск функции train_agent для обучения агента >

Детерминированная стратегия:

In [ ]:
record_sessions(agent, 'LunarLander-v2', t_max=1000)
show_video('LunarLander-v2', 'test')

Стохастическая стратегия:

In [ ]:
show_video('LunarLander-v2', 'sample')

### Ещё дополнительное задание (вплоть до 5 баллов)

* __2.3 дополнительное__ Обучить агента в среде с непрерывным пространством действий с помощью `MLPRegressor` ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor)) или похожим образом.
  * Начните с [Pendulum-v0](https://www.gymlibrary.dev/environments/classic_control/pendulum/), попробуйте получить среднее вознаграждение **как минимум -300**.
  * Поскольку Ваш агент оценивает "среднее" действие, полезно добавить небольшой шум для исследования среды.
  * Обучить агентов в [MountainCarContinuous-v0](https://mgoulao.github.io/gym-docs/environments/classic_control/mountain_car_continuous/), [LunarLanderContinuous-v2](https://www.gymlibrary.dev/environments/box2d/lunar_lander/). За достижение награды ниже порогового значения будет начислено меньше баллов. Помните, что дискретные и непрерывные среды могут отличаться не только в пространстве действий. Требования на среднее вознаграждение такие же, как и в случае сред **MountainCar** и **LunarLander** ранее.
  * __При сдаче задания требуется перечислить в данном файле, что было сделано и чего добились__

## Обучение в Pendulum-v0

In [ ]:
sns.set(font_scale=1.)
env, n_actions, _ = get_env('Pendulum-v0', False)

In [ ]:
env.observation_space, env.action_space

Обучение агента:

In [ ]:
%%time

agent, _ = < запуск функции train_agent для обучения агента >

Детерминированная стратегия:

In [ ]:
record_sessions(agent, 'Pendulum-v0', t_max=1000, classification=False, epsilon=.5, agent_mul_fac=2.)
show_video('Pendulum-v0', 'test')

Стохастическая стратегия:

In [ ]:
show_video('Pendulum-v0', 'sample')

## Обучение MountainCarContinuous-v0

In [ ]:
sns.set(font_scale=1.)
env, n_actions, _ = get_env('MountainCarContinuous-v0', False)

In [ ]:
env.observation_space, env.action_space

Обучение агента:

In [ ]:
%%time

agent, _ = < запуск функции train_agent для обучения агента >

Детерминированная стратегия:

In [ ]:
record_sessions(agent, 'MountainCarContinuous-v0', t_max=10000, classification=False, epsilon=1.0,
                agent_mul_fac=1.0)
show_video('MountainCarContinuous-v0', 'test')

Стохастическая стратегия:

In [ ]:
show_video('MountainCarContinuous-v0', 'sample')

## Обучение LunarLanderContinuous-v2

In [ ]:
sns.set(font_scale=1.)
env, n_actions, _ = get_env('LunarLanderContinuous-v2', False)

In [ ]:
env.observation_space, env.action_space

Обучение агента:

In [ ]:
%%time

agent, _ = < запуск функции train_agent для обучения агента >

Детерминированная стратегия:

In [ ]:
record_sessions(agent, 'LunarLanderContinuous-v2', t_max=1000, classification=False, epsilon=1.0,
                agent_mul_fac=1.0)
show_video('LunarLanderContinuous-v2', 'test')

Стохастическая стратегия:

In [ ]:
show_video('LunarLanderContinuous-v2', 'sample')